In [1]:
# THis is just a COMPUTE node!
# this has a LOCAL filesystem -> which is just a reference point
# all my data -> DataLake!
# LOAD my DataLake into Databricks!
#   -> MOUNTING 

In [2]:
# source is datalake, mount-> local name for this datalake to treat it like a
# local file system
# wasbs; http, ftp, and other protocols, wasb is the default communication 
# protocol for apache spark, wasbs -> secure for for it!
#dbutils.fs.mount(source='wasbs://containername@storageaccount', mount_point, extra_configs)
# giraffeisagovernmentconspiracy
# pikapikapikachu.blob.core.windows.net

# /mnt -> mnt is a folder which holds all the disk available to spark cluster

# extra configs: this is the name of reference that developer should use instead of 
# actual path or secret or connection string
# key: value
# key-> file system to actually access lake 
# value-> secret to access the file system

# key: fs.azure.sas.container
# value: secret

# previous syntax-> storage account (fs.azure.sas.)
# this syntax -> datalakes (fs.azure.account.key.)

dbutils.fs.mount(source='wasbs://giraffeisagovernmentconspiracy@pikapikapikachu.blob.core.windows.net', mount_point='/mnt/lake', extra_configs= {"fs.azure.account.key.pikapikapikachu.blob.core.windows.net":dbutils.secrets.get(scope='melody',key='mysecret')})

Out[ 6 ]: True

In [3]:
# because this file is locally present, i can access via sparkcontext!

df = spark.read.csv('/mnt/lake/cylons.csv', header=True)


In [4]:
df.show()

+------+-------+---+
 Fname| Lname|age|
+------+-------+---+
 Peter| Parker| 99|
Donald| Duck| 22|
 Peter|Griffin| 80|
+------+-------+---+

In [5]:
df.head()

Out[ 10 ]: Row(Fname='Peter', Lname='Parker', age='99')

In [6]:

# all data, and processing is happening in RAM
# if you data was 20GB, and ram is just 2 GB-> how do i even load?
# data size => every data point is stored at least thrice 
#    - BigQuery, BigTable, DataBricks, CosmosDB, DataLakes, S3
# planning for RAM-> at least THRICE than the data you need to process 


In [7]:
# we can use SQL statements without any external factor
mydata = df.select('Fname','age')
mydata.show()

+------+---+
 Fname|age|
+------+---+
 Peter| 99|
Donald| 22|
 Peter| 80|
+------+---+

In [8]:
renameddf = df.withColumnRenamed('Fname','name')
renameddf.show()


+------+-------+---+
 name| Lname|age|
+------+-------+---+
 Peter| Parker| 99|
Donald| Duck| 22|
 Peter|Griffin| 80|
+------+-------+---+

In [9]:
from pyspark import Row
# ROW => represents logic to create a SINGLE ROW in RDD 
# dummy records or read from external source like file or db!
records= [('Pikachu',55),('Yokozuna',20),('Scrooge',25),('gangadhar',40)]
rdd = sc.parallelize(records) # RDD's memory allocation request 
myData = rdd.map(lambda record: Row(name=record[0], age=record[1])) 
# a logic to create a ROW-> lambda function to logic

In [10]:
myDF = sqlContext.createDataFrame(myData)
myDF.show() 
# SQL -> order of column NEVER mattered in the first place!
# EVERY column is a new file by itself!!! 
# ORDER of your columns will not matter! 

+---+---------+
age| name|
+---+---------+
 55| Pikachu|
 20| Yokozuna|
 25| Scrooge|
 40|gangadhar|
+---+---------+

In [11]:
# load csv 
#alldata = sqlContext.load(source='com.databricks.spark.csv',path='/mnt/lake/cylons.csv', header=True, inferSchema=True)

df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/mnt/lake/data.csv')
df.printSchema()

# 4:15 please resume! 

root
-- id: integer (nullable = true)
-- diagnosis: string (nullable = true)
-- radius_mean: double (nullable = true)
-- texture_mean: double (nullable = true)
-- perimeter_mean: double (nullable = true)
-- area_mean: double (nullable = true)
-- smoothness_mean: double (nullable = true)
-- compactness_mean: double (nullable = true)
-- concavity_mean: double (nullable = true)
-- concave points_mean: double (nullable = true)
-- symmetry_mean: double (nullable = true)
-- fractal_dimension_mean: double (nullable = true)
-- radius_se: double (nullable = true)
-- texture_se: double (nullable = true)
-- perimeter_se: double (nullable = true)
-- area_se: double (nullable = true)
-- smoothness_se: double (nullable = true)
-- compactness_se: double (nullable = true)
-- concavity_se: double (nullable = true)
-- concave points_se: double (nullable = true)
-- symmetry_se: double (nullable = true)
-- fractal_dimension_se: double (nullable = true)
-- radius_worst: double (nullable = true)
-- texture_worst: double (nullable = true)
-- perimeter_worst: double (nullable = true)
-- area_worst: double (nullable = true)
-- smoothness_worst: double (nullable = true)
-- compactness_worst: double (nullable = true)
-- concavity_worst: double (nullable = true)
-- concave points_worst: double (nullable = true)
-- symmetry_worst: double (nullable = true)
-- fractal_dimension_worst: double (nullable = true)
-- _c32: string (nullable = true)

In [12]:
stats = df.describe('smoothness_mean')
stats.show()

+-------+--------------------+
summary| smoothness_mean|
+-------+--------------------+
 count| 569|
 mean| 0.096360281195079|
 stddev|0.014064128137673616|
 min| 0.05263|
 max| 0.1634|
+-------+--------------------+

In [13]:
x = df.select('smoothness_mean').subtract(df.select('smoothness_worst'))
# FORMER - LATTER 
# NUMPY , instead of nUMPY-> sql INSTEAD
x.head(5)

Out[ 32 ]: [Row(smoothness_mean=0.09754),
 Row(smoothness_mean=0.08182),
 Row(smoothness_mean=0.09592),
 Row(smoothness_mean=0.08999),
 Row(smoothness_mean=0.08597)]

In [14]:
x = df.crosstab('radius_mean','diagnosis')
# using crosstab, you can also check FREQ Distribution
x.show()

+---------------------+---+---+
radius_mean_diagnosis| B| M|
+---------------------+---+---+
 11.89| 3| 0|
 19.27| 0| 1|
 13.34| 1| 0|
 14.25| 0| 2|
 11.22| 2| 0|
 13.59| 2| 0|
 20.48| 0| 1|
 9.436| 1| 0|
 12.46| 2| 1|
 13.62| 1| 0|
 25.73| 0| 1|
 14.29| 1| 0|
 17.85| 1| 0|
 15.61| 0| 1|
 12.32| 1| 0|
 10.2| 1| 0|
 15.3| 0| 1|
 12.65| 1| 0|
 12.95| 1| 0|
 9.295| 1| 0|
+---------------------+---+---+
only showing top 20 rows

In [15]:
df.dropna().count() # drop NULL values 

Out[ 35 ]: 0

In [16]:
# GENERATE null values to handle in next block
records= [(None,55),('Yokozuna',20),('Scrooge',25),('gangadhar',40)]
rdd = sc.parallelize(records) # RDD's memory allocation request 
myData = rdd.map(lambda record: Row(name=record[0], age=record[1])) 
myDF = sqlContext.createDataFrame(myData)
myDF.show()

+---+---------+
age| name|
+---+---------+
 55| null|
 20| Yokozuna|
 25| Scrooge|
 40|gangadhar|
+---+---------+

In [17]:
df_null_replaced = myDF.fillna('No Name')
df_null_replaced.show()

+---+---------+
age| name|
+---+---------+
 55| No Name|
 20| Yokozuna|
 25| Scrooge|
 40|gangadhar|
+---+---------+

In [18]:
filteredDF = df.filter(df.smoothness_mean>0.1)
print(filteredDF.count())

216

In [19]:
df.groupby('diagnosis').count().show()

+---------+-----+
diagnosis|count|
+---------+-----+
 B| 357|
 M| 212|
+---------+-----+

In [20]:
df.groupby('diagnosis').agg({'radius_mean':'mean'}).show()
# Malignent cells have to be larger then benign cancer cells !


+---------+-----------------+
diagnosis| avg(radius_mean)|
+---------+-----------------+
 B|12.14652380952381|
 M|17.46283018867925|
+---------+-----------------+

In [21]:
# Python -> SAMPLE can be used to divide data into multiple dataframes
t1 = df.sample(False,0.2,42) # 20% data, random_state=43
t2 = df.sample(False,0.2,43) # 20% data, random_state=43
t3 = df.sample(False,0.2,44)
# we ensured that same shuffling doesn't result in same dataset! 
# there was some data overlapping! 
print(t1.count())
print(t2.count())
print(t3.count())


121
100
115

In [22]:
# drop columns ->ONLY after Exploratory Data Analysis
print(df.columns)
cols = df.drop('id').columns # it was a list over here
# cols = df.drop('id') # this would be a DataFrame 
print(cols)

['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', '_c32']
['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', '_c32']

In [23]:
df.registerTempTable('bc_table') # i can write QUERIES on the df, 
# as if it were a SQL 
x = sqlContext.sql('select radius_mean from bc_table')
x.show()

+-----------+
radius_mean|
+-----------+
 17.99|
 20.57|
 19.69|
 11.42|
 20.29|
 12.45|
 18.25|
 13.71|
 13.0|
 12.46|
 16.02|
 15.78|
 19.17|
 15.85|
 13.73|
 14.54|
 14.68|
 16.13|
 19.81|
 13.54|
+-----------+
only showing top 20 rows

In [24]:
#                Python       PySpark
# a=1              a=1          True
# b=2              b=2          True
# c=a+b            c=3          True
# print(c)         3            starts solving-> gives you result

In [25]:
print(df.columns)

['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', '_c32']

In [26]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

In [27]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.3 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell